## Pasos iniciales: 
**1. Importarmos los modulos pandas, numpy, y pyplot**

**2. Creamos la variable originalDf asignandole el DataSet**

**3. Mostramos los primeros 5 registros del DataFrame**

    Con el objetivo de tener un panorama general de entrada podemos observar que
    existen valores Nan en Province/State lo que nos permite notar
    dos posibles panoramas:

    Eliminar las columnas que son inecesarias para responder las preguntas planteadas? o

    Transformar estos valores NaN para contar con todos los datos? 

    Continuaremos la exploración para tomar decisiones...

In [10]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#para manipulacion de formato en español y ultima fecha del archivo
import locale 
import calendar  


In [11]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8') #Para poner el formato de mes en español

'es_ES.UTF-8'

In [12]:
originalDf = pd.read_csv('covid.csv')

In [13]:
originalDf.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


**4. Verificamos los registros totales, los registrs nulos de cada columna**

    Tenemos 289 registros (filas) y 1147 columnas    
    Verificamos tambien con .info() el tipo DataType de cada columna encontrando que:
    Province/State y Country/Region son variables categoricas 
    Lat y Long son variables float64
    Y las fechas son númericas (deberian ser Date)

    Hasta el momento podremos comentar que las fechas deberían modificarse a tipo Date para lograr una correcta manipulacion de los datos
    de cara a responder las preguntas que se han planteado en el proyecto 


In [15]:
originalDf.shape

(289, 1147)

In [16]:
originalDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Columns: 1147 entries, Province/State to 3/9/23
dtypes: float64(2), int64(1143), object(2)
memory usage: 2.5+ MB


originalDf.isna().sum()

**Basados en las preguntas:**

1. ¿En cuál mes se presentó el mayor número de contagios?
2. ¿En ese mismo mes, cuál fue el país que reportó más contagios?
3. ¿Cuál es el país con el menor número de casos reportados hasta la
fecha?

**5. Analizamos  que:**

    -Las columnas **Province/State y Lat y Long** no son de importancia para responder las preguntas planteadas
    -Las fechas como evidenciamos antes deben ser llevadas a formato dateTime
    -El mes necesita ser separado para el posterior analisis 
    -Ademas encontramos que el valor de la columna contagios es acumulativo
    
    Entonces: 
    
    -Eliminamos las columnas **Province/State y Lat y Long**
    
    -Reorganizamos el DataFrame para convertir las columnas de fechas en una columna única
    llamada **Fecha** con formato **datetime** con los valores de **Contagios** en otra    columna
    
    -Cambiamos el nombre de la columna **Country/Region** a **Pais**
    
    -Separamos el mes de cada fecha y lo guardamos en una columna mes 

    -Para resolver el problema de la columna acumulativa vamos a obtener el total de contagios
    diarios restando el valor acumulado del dia anterior .diff()
    
    Para ello usaremos una copia de nuestro DF original en df_toclean


In [19]:
df_toclean = originalDf.copy()

In [21]:
df_toclean.drop(columns=['Province/State', 'Lat', 'Long'], inplace=True)
df_toclean.isna().sum()

Country/Region    0
1/22/20           0
1/23/20           0
1/24/20           0
1/25/20           0
                 ..
3/5/23            0
3/6/23            0
3/7/23            0
3/8/23            0
3/9/23            0
Length: 1144, dtype: int64

In [23]:
df_melted = df_toclean.melt(id_vars=["Country/Region"], var_name="Fecha", value_name="Contagios")


In [26]:
df_toclean = df_toclean[~df_toclean['Country/Region'].isin(["Winter Olympics 2022", "Summer Olympics 2020",'Diamond Princess', 'MS Zaandam'])]


Cambiamos el nombre 

In [31]:
df_melted['Fecha'] = pd.to_datetime(df_melted['Fecha'], format='%m/%d/%y')


Ordenamos los datos por País y Fecha para calcular a diario

Calculamos el número de contagios diarios restando el valor acumulado del dia anterior
Hacemos uso del metodo .diff()

In [37]:

df_melted = df_melted.sort_values(by=['Country/Region', 'Fecha'])


In [39]:
df_melted['ContagiosDiarios'] = df_melted.groupby('Country/Region')['Contagios'].diff().fillna(0)

In [41]:
df_melted['Mes'] = df_melted['Fecha'].dt.month
df_melted['Año'] = df_melted['Fecha'].dt.year


**Pensando en que estos registros no representan información de paises**

    Eliminamos los  registros "Winter Olympics 2022", "Summer Olympics 2020" ,'Diamond Princess', 'MS Zaandam

In [44]:
df_melted.rename(columns={"Country/Region": "Pais"}, inplace=True)

**6. Finalmente respondemos las preguntas:**


In [61]:
contagios_por_mes_año = df_melted.groupby(['Año', 'Mes'])['ContagiosDiarios'].sum()
mes_año_max_contagios = contagios_por_mes_año.idxmax()
max_contagios = contagios_por_mes_año.max()
nombre_mes_max_contagios = calendar.month_name[mes_año_max_contagios[1]].capitalize()

print(f"El mes con el mayor número de contagios es: {nombre_mes_max_contagios} del año {mes_año_max_contagios[0]}, con {max_contagios} contagios.")


El mes con el mayor número de contagios es: Enero del año 2022, con 87136808.0 contagios.


In [63]:
# Pregunta 2: ¿En ese mismo mes y año, cuál fue el país que reportó más contagios y cuántos fueron?
df_mes_año_max = df_melted[(df_melted['Mes'] == mes_año_max_contagios[1]) & (df_melted['Año'] == mes_año_max_contagios[0])]
pais_max_contagios = df_mes_año_max.groupby('Pais')['ContagiosDiarios'].sum().idxmax()
contagios_pais_max = df_mes_año_max.groupby('Pais')['ContagiosDiarios'].sum().max()
print(f"En {mes_año_max_contagios[1]}/{mes_año_max_contagios[0]}, el país con más contagios fue {pais_max_contagios}, con {contagios_pais_max} contagios.")


En 1/2022, el país con más contagios fue US, con 20336435.0 contagios.


In [65]:
# Pregunta 3: ¿Cuál es el país con el menor número de casos reportados hasta la fecha?
contagios_totales_por_pais = df_melted.groupby('Pais')['ContagiosDiarios'].sum()
pais_min_contagios = contagios_totales_por_pais.idxmin()
contagios_pais_min = contagios_totales_por_pais.min()
print(f"El país con el menor número de casos reportados es: {pais_min_contagios}, con {contagios_pais_min} contagios.")

El país con el menor número de casos reportados es: Korea, North, con 1.0 contagios.


Verificamos las columnas y que no existan valores null ademas usamos .sample() para visualizar el formato del df_melted

In [54]:
df_melted.info()


<class 'pandas.core.frame.DataFrame'>
Index: 330327 entries, 0 to 330326
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Pais              330327 non-null  object        
 1   Fecha             330327 non-null  datetime64[ns]
 2   Contagios         330327 non-null  int64         
 3   ContagiosDiarios  330327 non-null  float64       
 4   Mes               330327 non-null  int32         
 5   Año               330327 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(1), object(1)
memory usage: 15.1+ MB


In [56]:
df_melted.head(100)

,Pais,Fecha,Contagios,ContagiosDiarios,Mes,Año
0,Afghanistan,2020-01-22,0,0.0,1,2020
289,Afghanistan,2020-01-23,0,0.0,1,2020
578,Afghanistan,2020-01-24,0,0.0,1,2020
867,Afghanistan,2020-01-25,0,0.0,1,2020
1156,Afghanistan,2020-01-26,0,0.0,1,2020
...,...,...,...,...,...,...
27455,Afghanistan,2020-04-26,1463,133.0,4,2020
27744,Afghanistan,2020-04-27,1531,68.0,4,2020
28033,Afghanistan,2020-04-28,1703,172.0,4,2020
28322,Afghanistan,2020-04-29,1827,124.0,4,2020


In [58]:
#df_melted.to_csv('covid_melted.csv', index=False)
#df_melted.to_excel('covid_melted.xlsx', index=False)

#### Investigación: 
**melt()**: Se utiliza para reestructurar datos. Convierte columnas en filas

**loc[]**: Se utiliza para acceder a filas y columnas especificas por etiquetas o condiciones

**get_level_values()**: Se utiliza para obtener valores especificos en DataFrames con índices  
jerárquicos, despues de agrupar valores

**max() min()**: Se utilizan para obtener maximo y/o minimo de una columna o DataFrame

**drop**:  Eliminar columnas 



**Conclusión:**
En el ambito de la Ciencia de Datos la biblioteca Pandas se ha convertido en una herramienta esencial. Desde su creacion ha facilitado el análisis y la manipulación de datos de formas que antes eran complicadas.

Antes de su creacion los cientificos de datos dependian de herramientas menos eficaces como Exel, R y SQL que aunque siendo poderesos requerian mayores esfuerzos comparados con Pandas ademas de hacer del analisis de datos una tarea tediosa, frustrante y muy demandante de tiempo por la limpieza y transformacion de los datos.

En 2008 Wes McKinney desarrollo Pandas mientras trabajaba en el análisis de datos en AQR Capital Management. Su objetivo era crear una herramienta que facilitara la manipulación de los datos, en su caso especialmente financieros, Pandas ha venido evolucionando constantemente desde aquel dia, introduccion de estructuras como series y DataFrames, compatibilidad con datos mas complejos y rendimiento.

Entre los beneficios de Pandas encontramos, facilidad de uso, manejo de datos faltantes, analisis eficientes y obviamente su impacto en el almacenamiento de Datos en DataLakes.

Impacto de Pandas en el manejo de DataLakes:
En la era del BigData los DataLakes se han convertido en una solucion popular para almacenar grandes volumenes de datos en su forma original. Esto permite a las organizaciones conservar datos sin una estructura rigida, lo que resulta útil para el análisis posterior.Aqui es donde Pandas muestra su verdadero valor al facilitar el trabajo en estos entornos:

Que es un DataLAke?
Un DataLake es un repositorio que permite almacenar datos de diferentes tipos(estructurados, semiestructurados y no estructurados) de manera eficiente y escalable. A diferencia de las bases de datos tradicionales, un DataLake no requiere que los datos sean organizados en tablas antes de su almacenamiento, lo que brinda mayor flexibilidad. Sin embargo, esta felxibilidad tambien puede presentar desafios a la hora de acceder y analizar los datos.

Rol de Pandas en DataLakes
Limpieza y transformación de Datos:
Antes de almacenar en un DL es esencial limpiar y transformar. Pamdas permite a los usuarios eliminar datos erroneos, rellenar valores faltantes y estandarizar formatos. Este proceso garantiza que los datos almacenados sean de alta calidad, lo que es fundamental para analisis futuros.

Preparación para el Análisis:
Una vez que los datos estan en DL, la preparación para el análisis puede ser complicada si los datos no estan bien organziados. Usando Pandas, los analistas pueden manipular los datos desde el DL y estructurarlos de forma adecuada antes de realizar cualquier análisis. Esto incluye actividades como la agregación de datos, la creación de subconjuntos y la fusion de diferentes conjuntos de datos.

Facilitación del acceso a la información:
Con los datos adecuadamente organizados y limpios, Pandas facilita el acceso a la información relevante. Esto significa que los analistas pueden ejecutar consultas y generar informes más rápidamente, lo que acelera la toma de decisiones de la organización

Optimizacion y rendimiento:
Pandas ayuda a reducir el tiempo de procesamiento necesario para analizar datos almacenados en un DL. Al realizar transformaciones y análisis previos a la carga de datos, los usuarios pueden evitar procesos pesados y complejos que ralentizan el acceso a la información.

La combinación de Pandas y DataLakes proporciona entornos robustos y eficientes para el manejo de grandes volumenes de datos

Fuentes
McKinney, W. (2010). Data Analysis in Python. Retrieved from Pandas Documentation.

Wes McKinney. (2009). Pandas: A Foundational Library for Data Analysis in Python. Retrieved from Pandas Documentation.

McKinney, W. (2020). Pandas 1.0.0 Release Notes. Retrieved from Pandas Documentation.